In [10]:
import sys
from multistrand.objects import *
from multistrand.options import Options, EnergyType, Literals
from multistrand.system import SimSystem, calculate_energy

In [11]:
def print_trajectory(o):
    seqstring=''
    for i in range(len(o.full_trajectory)): # go through each output microstate of the trajectory
        time = o.full_trajectory_times[i]   # time at which this microstate is entered
        states = o.full_trajectory[i]       # this is a list of the complexes present in this tube microstate
        newseqs = []
        for state in states: newseqs += [ state[3] ]   # extract the strand sequences in each complex (joined by "+" for multistranded complexes)
        newseqstring = ' '.join(newseqs)    # make a space-separated string of complexes, to represent the whole tube system sequence
        if not newseqstring == seqstring :
            print(newseqstring)
            seqstring=newseqstring          # because strand order can change upon association of dissociation, print it when it changes
        structs = []
        for state in states: structs += [ state[4] ]   # similarly extract the secondary structures for each complex
        tubestruct = ' '.join(structs)      # give the dot-paren secondary structure for the whole test tube
        dG=0
        for state in states: dG += state[5]
        print('%s t=%11.9f seconds, dG=%6.2f kcal/mol' % (tubestruct,time, dG))

        # Needlessly verify that the reported trajectory energies are the Tube_Energy values
        dGv=0
        for state in states:
            cs=state[3].split('+')
            st=state[4]
            dGv += calculate_energy( [Complex( strands=[Strand(sequence=s) for s in cs], structure=st)], o, EnergyType.tube)[0]
        if not dGv == dG: print("Energy Mismatch")

In [46]:
def create_setup():
    toehold_length = 7
    toehold_seq = 'ATGTGGAGGG'
    bm_design = "GGTGAGTTTGAGGTTGA"
    incum_extra_design = "TGGTGTTTGTGGGTGT"

    # creating Domain objects
    toehold = Domain(name="toehold", sequence=toehold_seq[0:toehold_length])
    branch_migration = Domain(name="bm", sequence=bm_design)

    # invader
    invader = branch_migration + toehold
    invader.name = "invader"

    # target (substrate)
    toehold_extra = Domain(name="toehold_extra", sequence=toehold_seq[toehold_length:]) 
    target = toehold_extra.C + toehold.C + branch_migration.C
    target.name = "target"

    # incumbent
    incumbent_extra = Domain(name="incumbent_extra", sequence=incum_extra_design)
    incumbent = incumbent_extra+branch_migration
    incumbent.name = "incumbent"

    # creates the target-incumbent start complex  
    start_complex_substrate_incumbent = Complex(strands=[target, incumbent], structure="..(+.)")
    # creates invader complex. 
    start_complex_incoming = Complex(strands=[invader], structure="..") 

    # creates a complex for a "succcessful displacement" stop condition. This is the incumbent strand forming a complex of its own which means it has been displaced.
    complete_complex_success = Complex(strands=[incumbent], structure="..")
    # success_stop_condition = StopCondition("SUCCESS", [(complete_complex_success, Literals.dissoc_macrostate, 0)])
    success_stop_condition = StopCondition("SUCCESS", [(complete_complex_success, Literals.exact_macrostate, 0)])
    
    

    # # complex to create failed displacement stop condition; incumbent falls off.   
    # failed_complex = Complex(strands=[invader], structure="..")  
    # failed_stop_conditions = StopCondition("FAILURE", [(failed_complex, Literals.dissoc_macrostate, 0)]) 
        
        
    o = Options(
        simulation_mode="Trajectory",   #  First Step
        substrate_type="DNA",
        num_simulations=1, 
        simulation_time=float('inf'),  # note the float('inf') second simulation time, to make sure simulations finish
        dangles="Some", 
        temperature=4, 
        join_concentration = 100e-6, # 100 uM
        gt_enable = False,
        output_interval = 1, # record every 1000 steps
        verbosity=0,
        start_state = [start_complex_incoming, start_complex_substrate_incumbent],
        # stop_conditions = [success_stop_condition, failed_stop_condition]
        stop_conditions = [success_stop_condition]
        )
    
    o.DNA23Metropolis()

    return o

In [47]:
stdoutOrigin=sys.stdout 
sys.stdout = open(f"./output_trajmodel/debug-exact.txt", "w")

for i in range(100):
    o = create_setup()
    s = SimSystem(o)
    s.start()
 
    print(len(o.full_trajectory))
    print(o.full_trajectory[-1])

    sys.stdout.flush()  # Flush the output here
    
sys.stdout.close()
sys.stdout=stdoutOrigin

In [44]:
len(o.full_trajectory), o.full_trajectory[-1]

(1677967,
 [(-4036093479859725470,
   5,
   '751674incumbent',
   'TGGTGTTTGTGGGTGTGGTGAGTTTGAGGTTGA',
   '.................................',
   0.0,
   0.0),
  (-4036093479859725470,
   4,
   '751673invader,751674target',
   'GGTGAGTTTGAGGTTGAATGTGGA+CCCTCCACATTCAACCTCAAACTCACC',
   '.(((((((((((((((((((((((+...))))))))))))))))))))))).',
   -40.085935321472945,
   -189.62123972271485)])

In [30]:
stdoutOrigin=sys.stdout 
# sys.stdout = open(f"./output_trajmodel/test_interval=1-exact2.txt", "w")
sys.stdout = open(f"./output_trajmodel/test_interval=1.txt", "w")

print_trajectory(o)
sys.stdout.flush()  # Flush the output here
sys.stdout.close()
sys.stdout=stdoutOrigin

In [15]:
len(o.full_trajectory)

468358

In [20]:
file_name = f'./raw_data/Machinek-PRF-og/Machinek-PRF-{3}.txt'
with open(file_name, 'r') as f:
    lines = f.readlines()
    
lines[756]

if 'Energy Mismatch' in lines[756]:
    print  ('yes')

'Energy Mismatch\n'

In [26]:
# find the files that have "Energy Mismatch" occurred

mismatch_files = []

for i in range(100):
    file_name = f'./raw_data/Machinek-PRF-og/Machinek-PRF-{i}.txt'
    
    with open(file_name, 'r') as f:
        lines = f.readlines()
    
    for line in lines:
        if "Energy Mismatch" in line:
            mismatch_files.append(i)
            break    

In [27]:
mismatch_files

[0,
 2,
 3,
 5,
 7,
 8,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 23,
 24,
 26,
 27,
 28,
 31,
 33,
 34,
 35,
 36,
 39,
 40,
 41,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 54,
 55,
 56,
 57,
 58,
 60,
 62,
 63,
 65,
 66,
 67,
 68,
 69,
 70,
 72,
 73,
 74,
 76,
 77,
 80,
 81,
 82,
 84,
 85,
 87,
 88,
 90,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [ ]:
[0, 2, 3, 5, 7, 8]
